# The Boston Housing Assess Dataset

Part III: Using Prepared Data to Train many models
In this Notebook, we use prepared data to train different models and compare their performances. 
The data used can be before or after feature selection work as well as from different years. 
Multi-year combined data are not tested yet. 

In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from IPython.core.display import HTML
from pandas import read_csv

In [2]:
from time import time
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import QuantileTransformer


In [8]:
myDataPath = 'M:\\Work\\Study\\WPI-DS\\CurrentCourse-DS504\\DataSets\\SelectedDataSet\\'
subFolder = 'BigDataset_Preprocess/'
fileName = 'BH_BigDataset_2019.csv'
fileBigData = myDataPath + subFolder + fileName
print('Input data file:\n', fileBigData)

Input data file:
 M:\Work\Study\WPI-DS\CurrentCourse-DS504\DataSets\SelectedDataSet\BigDataset_Preprocess/BH_BigDataset_2019.csv


In [9]:
#Lets load the dataset and sample some
data = read_csv(fileBigData)
#print(data.head(5))

In [10]:
# Dimension of the dataset
print(np.shape(data))

(159937, 46)


In [20]:
df = data
data = df.replace(np.nan, '', regex=True)
display(HTML(data.head().to_html()))

,LU,OWN_OCC,AV_TOTAL,YR_BUILT,YR_REMOD,LIVING_AREA,NUM_FLOORS,R_BLDG_STYL,R_ROOF_TYP,R_EXT_FIN,R_BDRMS,R_FULL_BTH,R_HALF_BTH,R_BTH_STYLE,R_BTH_STYLE2,R_BTH_STYLE3,R_KITCH,R_KITCH_STYLE,R_KITCH_STYLE2,R_KITCH_STYLE3,R_HEAT_TYP,R_AC,R_FPLACE,R_EXT_CND,R_OVRALL_CND,R_INT_CND,R_INT_FIN,R_VIEW,U_BASE_FLOOR,U_NUM_PARK,U_CORNER,U_ORIENT,U_BDRMS,U_FULL_BTH,U_HALF_BTH,U_BTH_STYLE,U_BTH_STYLE2,U_BTH_STYLE3,U_KITCH_TYPE,U_KITCH_STYLE,U_HEAT_TYP,U_AC,U_FPLACE,U_INT_FIN,U_INT_CND,U_VIEW
0,CD,Y,13.685631,7.523481,2006.0,6.672033,0.693147,NaN,NaN,NaN,1.678303,1.074549,0.247201,NaN,NaN,NaN,1.748661,NaN,NaN,NaN,NaN,NaN,0.229554,NaN,NaN,NaN,NaN,NaN,1.098612,0.693147,N,F,0.693147,0.693147,0.693147,M,M,NaN,O,M,W,N,0.693147,N,G,A
1,CD,N,13.614130,7.523481,1981.0,6.624065,0.693147,NaN,NaN,NaN,1.678303,1.074549,0.247201,NaN,NaN,NaN,1.748661,NaN,NaN,NaN,NaN,NaN,0.229554,NaN,NaN,NaN,NaN,NaN,1.098612,0.693147,N,A,0.693147,0.693147,0.000000,S,NaN,NaN,O,S,W,N,0.693147,N,A,A
2,CD,Y,13.652404,7.523481,2016.0,6.675823,0.693147,NaN,NaN,NaN,1.678303,1.074549,0.247201,NaN,NaN,NaN,1.748661,NaN,NaN,NaN,NaN,NaN,0.229554,NaN,NaN,NaN,NaN,NaN,1.386294,0.000000,N,F,1.098612,1.098612,0.000000,M,S,NaN,O,M,W,N,0.693147,N,A,G
3,CD,N,13.506810,7.523481,1981.0,6.580639,0.693147,NaN,NaN,NaN,1.678303,1.074549,0.247201,NaN,NaN,NaN,1.748661,NaN,NaN,NaN,NaN,NaN,0.229554,NaN,NaN,NaN,NaN,NaN,1.386294,0.693147,N,A,0.693147,0.693147,0.000000,N,NaN,NaN,O,N,W,N,0.693147,S,A,G
4,CD,Y,14.586683,7.523481,2014.0,7.301822,0.693147,NaN,NaN,NaN,1.678303,1.074549,0.247201,NaN,NaN,NaN,1.748661,NaN,NaN,NaN,NaN,NaN,0.229554,NaN,NaN,NaN,NaN,NaN,1.609438,1.098612,N,F,1.098612,1.098612,0.693147,L,M,M,O,L,W,C,1.098612,N,E,G


In [22]:
# Find all categorical features that used as input to CatBoost model
catFeatList = data.dtypes[data.dtypes == "object"].index
print(len(catFeatList))
print(catFeatList)

30
Index(['LU', 'OWN_OCC', 'R_BLDG_STYL', 'R_ROOF_TYP', 'R_EXT_FIN',
       'R_BTH_STYLE', 'R_BTH_STYLE2', 'R_BTH_STYLE3', 'R_KITCH_STYLE',
       'R_KITCH_STYLE2', 'R_KITCH_STYLE3', 'R_HEAT_TYP', 'R_AC', 'R_EXT_CND',
       'R_OVRALL_CND', 'R_INT_CND', 'R_INT_FIN', 'R_VIEW', 'U_CORNER',
       'U_ORIENT', 'U_BTH_STYLE', 'U_BTH_STYLE2', 'U_BTH_STYLE3',
       'U_KITCH_TYPE', 'U_KITCH_STYLE', 'U_HEAT_TYP', 'U_AC', 'U_INT_FIN',
       'U_INT_CND', 'U_VIEW'],
      dtype='object')


In [23]:
x = data.loc[:,data.columns != 'AV_TOTAL']
y = data['AV_TOTAL']
print(data.shape)
print(x.shape)
print(y.shape)

(159937, 46)
(159937, 45)
(159937,)


In [24]:
print(x.shape)
print(y.shape)

(159937, 45)
(159937,)


In [25]:
#Split data for model training
fSplit = 0.1
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=fSplit, random_state=6)

#prepared for training model using split

In [26]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Run this after model training, using estimator, it can calculated multiple scores for test data
def CalcScore(est):
    y1_test = est.predict(X_test)
    # Three scores are calculated below
    R2 = r2_score(y_test, y1_test)
    print('R2 score: ', R2)
    mae = mean_absolute_error(y_test, y1_test)
    print('MAE score: ', mae)
    mse = mean_squared_error(y_test, y1_test)
    print('MSE score: ', mse)

In [27]:
from catboost import CatBoostRegressor

print("Training CatBoostRegressor...")
ticS = time()
est = CatBoostRegressor(verbose=0, cat_features=catFeatList, n_estimators=100)
est.fit(X_train, y_train)
ticE = time()
print("Fit done in {:.4f}s".format(ticE - ticS))
print("Test R2 score: {:.4f}".format(est.score(X_test, y_test)))
CalcScore(est)

Training CatBoostRegressor...
Fit done in 17.3351s
Test R2 score: 0.7529
R2 score:  0.752919645437685
MAE score:  0.3146992376617967
MSE score:  0.26246779734290876


In [30]:
# Collect results in the following map
scores_map = {}
scores_map_emsemble = {}

#from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(x)

nSplits = 10
kf = KFold(n_splits=nSplits)

#This one is used for cross validation scoring
customScoring = 'neg_mean_squared_error'
scoreStr = 'MSE'

print('Split: ', nSplits)
print('Custom scoring type: ', scoreStr)

Split:  10
Custom scoring type:  MSE


In [31]:
# Model 9: CT
algoName = 'CatBoostRegressor'
print("Training %s..." % algoName)
ticS = time()

cbr = CatBoostRegressor(verbose=0, cat_features=catFeatList, n_estimators=100)
scores = cross_val_score(cbr, x, y, cv=kf, scoring=customScoring)
ticE = time()
scores_map[algoName] = scores
scores_map_emsemble[algoName] = scores
print("{} CV done in {:.4f}s".format(algoName, ticE - ticS))
print("%s: %0.4f (+/- %0.4f)" % (scoreStr, scores.mean(), scores.std()))

#Training CatBoostRegressor...
#CatBoostRegressor CV done in 74.9145s
#MSE: -0.2875 (+/- 0.0664)

Training CatBoostRegressor...
CatBoostRegressor CV done in 216.2836s
MSE: -0.3026 (+/- 0.0843)
